# 서울 부동산 시장 분석 및 투자 전략 수립

## 데이터 전처리

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

df_sales_raw_2020 = pd.read_csv('./data/아파트(매매)_실거래가_2020.csv', encoding='EUC-KR')
df_sales_raw_2021 = pd.read_csv('./data/아파트(매매)_실거래가_2021.csv', encoding='EUC-KR')
df_sales_raw_2022 = pd.read_csv('./data/아파트(매매)_실거래가_2022.csv', encoding='EUC-KR')
df_sales_raw_2023 = pd.read_csv('./data/아파트(매매)_실거래가_2023.csv', encoding='EUC-KR')
df_sales_raw_2024 = pd.read_csv('./data/아파트(매매)_실거래가_2024.csv', encoding='EUC-KR')
df_sales_raw_2025 = pd.read_csv('./data/아파트(매매)_실거래가_2025.csv', encoding='EUC-KR')

df_sales_raw = pd.concat([df_sales_raw_2020, df_sales_raw_2021, df_sales_raw_2022, df_sales_raw_2023, df_sales_raw_2024, df_sales_raw_2025])

In [17]:
# 데이터 정리
df_sales = pd.DataFrame({
    '구': df_sales_raw['시군구'].str.split(' ').str[1],
    '동': df_sales_raw['시군구'].str.split(' ').str[2],
    '전용면적': df_sales_raw['전용면적(㎡)'],
    '계약일': pd.to_datetime(df_sales_raw['계약년월'].astype(str) + df_sales_raw['계약일'].astype(str).str.zfill(2)),
    '건축년도': df_sales_raw['건축년도'],
    '단지명': df_sales_raw['단지명'],
    '구분': '매매',
    '거래금액': df_sales_raw['거래금액(만원)'].str.replace(',', '').astype(int) * 10000,
    '매수자': df_sales_raw['매수자'],
    '매도자': df_sales_raw['매도자'],
    '해제사유발생일': df_sales_raw['해제사유발생일']
})
print(f'전체거래: {len(df_sales)}건')

# 취소된 거래 제거
valid_mask = df_sales['해제사유발생일'] == '-'
print(f'취소거래: {len(df_sales[~valid_mask])}건')

df_sales = df_sales[valid_mask]
df_sales = df_sales.drop('해제사유발생일', axis=1)

# 공공기관이 구매한 데이터(임대주택 등) 제거 - 과거 데이터에는 매수자/매도자 없음
# 월별로 같은 동에서 20건 이상 거래가 있을 경우는 의심거래로 분류
monthly_counts = df_sales.groupby(['단지명', '계약일'])['구'].transform('count')
df_sales.loc[monthly_counts >= 20, '공공임대'] = True

# 2024년 이후 데이터에서는 공공기관이 매수한 경우는 의심거래로 분류
df_sales.loc[df_sales['매수자'] == '공공기관', '공공임대'] = True

print(f'공공임대: {df_sales['공공임대'].sum()}건')

# csv로 저장
df_sales.to_csv('./data/sales_clean.csv', index=False)
display(df_sales.head())

전체거래: 279784건
취소거래: 13496건
공공임대: 3611건


,구,동,전용면적,계약일,건축년도,단지명,구분,거래금액,매수자,매도자,공공임대
0,성북구,돈암동,84.980,2020-12-31,2013,돈암동해피트리,매매,820000000,-,-,NaN
1,용산구,한남동,240.305,2020-12-31,2011,한남더힐,매매,7100000000,-,-,NaN
2,성동구,금호동4가,84.880,2020-12-31,2018,힐스테이트서울숲리버,매매,1800000000,-,-,NaN
3,동대문구,제기동,104.220,2020-12-31,1978,공성,매매,595000000,-,-,NaN
4,용산구,도원동,84.920,2020-12-31,2001,삼성래미안,매매,1350000000,-,-,NaN
